In [23]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [24]:
dataPr = pd.read_csv('dataset_pr.csv')

In [25]:
dataPr

,Tweet,HS,Abusive
0,cowok usaha lacak perhati lantas remeh perhati...,1,1
1,terlambat eluedan sarap gaul cigax jifla cal n...,0,1
2,kadang berfikir percaya tuhan jatuh berkalikal...,0,0
3,akunnku mata sipit lihat,0,0
4,kaum cebong kapir lihat dongok dongok hahahah,1,1
...,...,...,...
13164,bicara ndasmu mulut kate anjyng,1,1
13165,kasur enak kunyuk,0,1
13166,hati hati bisu gnnlagi bosan mengeluh xfxfxxaa,0,0
13167,bom real mudah deteksi bom kubur dahsyat ledak...,0,0


In [26]:
dataPr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13169 entries, 0 to 13168
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Tweet    13125 non-null  object
 1   HS       13169 non-null  int64 
 2   Abusive  13169 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 308.8+ KB


In [27]:
dataPr.isnull().sum()

Tweet      44
HS          0
Abusive     0
dtype: int64

In [28]:
dataPr = dataPr.dropna()

VECTORIZEN

In [30]:
X = dataPr['Tweet']
y = dataPr[['HS']]

In [34]:
from tensorflow.keras.layers import TextVectorization

In [35]:
MAX_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1800

In [36]:
vectorizer = TextVectorization(max_tokens=MAX_WORDS,
                               output_sequence_length=1800,
                               output_mode = 'int')

In [37]:
vectorizer.adapt(X.values)

In [38]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'jokowi',
 'presiden',
 'agama',
 'islam',
 'indonesia',
 'cebong',
 'lihat',
 'cina',
 'banget',
 'rakyat',
 'gubernur',
 'asing',
 'negara',
 'pakai',
 'kepala',
 'rezim',
 'daerah',
 'bicara',
 'kristen',
 'kerja',
 'komunis',
 'gantipresiden',
 'ekonomi',
 'dukung',
 'ahok',
 'pki',
 'salah',
 'bodoh',
 'anak',
 'amp',
 'budaya',
 'deh',
 'allah',
 'ulama',
 'anjing',
 'bilang',
 'kafir',
 'pemilihan',
 'pilih',
 'suka',
 'perintah',
 'org',
 'prabowo',
 'isu',
 'emang',
 'pei',
 'masuk',
 'nyata',
 'nama',
 'antek',
 'muslim',
 'laku',
 'bikin',
 'politik',
 'ajar',
 'anti',
 'xfxfxx',
 'kaya',
 'gitu',
 'kayak',
 'biar',
 'bom',
 'pimpin',
 'onta',
 'dr',
 'babi',
 'makan',
 'banci',
 'mata',
 'otak',
 'katolik',
 'hidup',
 'dasar',
 'yahudi',
 'umat',
 'ganti',
 'jalan',
 'hati',
 'aseng',
 'bohong',
 'kasih',
 'dungu',
 'lengser',
 'sialan',
 'sdh',
 'nusantara',
 'bubar',
 'beda',
 'sipit',
 'cinta',
 'nista',
 'manusia',
 'bani',
 'main',
 'paai',
 'bangsa',
 

In [39]:
vectorized_text = vectorizer(X.values)

In [40]:
vectorized_text

<tf.Tensor: shape=(13125, 1800), dtype=int64, numpy=
array([[ 679,  203, 3971, ...,    0,    0,    0],
       [1647,    1,  307, ...,    0,    0,    0],
       [ 612, 2425,  112, ...,    0,    0,    0],
       ...,
       [  79,   79,  293, ...,    0,    0,    0],
       [  63, 2822,  302, ...,    0,    0,    0],
       [ 473,  318, 1589, ...,    0,    0,    0]], dtype=int64)>

In [41]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [42]:
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X.values)

In [43]:
sequences = tokenizer.texts_to_sequences(X.values)
padded = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [44]:
import pickle

### Data Pipeline

In [46]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(13125)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [47]:
batch_X, batch_y = dataset.as_numpy_iterator().next()

In [48]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [49]:
train_generator = train.as_numpy_iterator()

In [50]:
train_generator.next()

(array([[  601,     1,   601, ...,     0,     0,     0],
        [  202,  2804,    41, ...,     0,     0,     0],
        [   19, 14420,   185, ...,     0,     0,     0],
        ...,
        [  357,     1,     1, ...,     0,     0,     0],
        [  147,   135,  2331, ...,     0,     0,     0],
        [ 1648,     2,   388, ...,     0,     0,     0]], dtype=int64),
 array([[0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1]], dtype=int64))

### Create Sequential Model

In [51]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [52]:
model = Sequential()
model.add(Embedding(input_dim=MAX_WORDS + 1, output_dim=32, input_length=MAX_SEQUENCE_LENGTH)) # Embedding Layer
model.add(Bidirectional(LSTM(32, activation='tanh'))) # Bidirectional LSTM Layer

# Feature extractor Fully connected Layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

# Output Layer
model.add(Dense(1, activation='sigmoid'))

c:\Users\Abiyoso\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [53]:
model.compile(loss='BinaryCrossentropy', optimizer='adam', metrics=['accuracy'])

In [54]:
model.build(input_shape=(None, 1800))

In [55]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 1800, 32)       │       640,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 64)             │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 731,041 (2.79 MB)

 Trainable params: 731,041 (2.79 MB)

 Non-trainable params: 0 (0.00 B)

In [56]:
from tensorflow.keras.callbacks import EarlyStopping

In [57]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,           
    restore_best_weights=True
)

In [ ]:
model.fit(
    train,
    validation_data=val,
    epochs=50,
    callbacks=[early_stop]
)

Epoch 1/5


574/574 ━━━━━━━━━━━━━━━━━━━━ 199s 343ms/step - accuracy: 0.6824 - loss: 0.5656 - val_accuracy: 0.8937 - val_loss: 0.2744
Epoch 2/5
574/574 ━━━━━━━━━━━━━━━━━━━━ 184s 320ms/step - accuracy: 0.8785 - loss: 0.2908 - val_accuracy: 0.9318 - val_loss: 0.1842
Epoch 3/5
574/574 ━━━━━━━━━━━━━━━━━━━━ 187s 325ms/step - accuracy: 0.9255 - loss: 0.1975 - val_accuracy: 0.9524 - val_loss: 0.1590
Epoch 4/5
574/574 ━━━━━━━━━━━━━━━━━━━━ 183s 318ms/step - accuracy: 0.9462 - loss: 0.1511 - val_accuracy: 0.9653 - val_loss: 0.1056
Epoch 5/5
574/574 ━━━━━━━━━━━━━━━━━━━━ 203s 354ms/step - accuracy: 0.9637 - loss: 0.1062 - val_accuracy: 0.9638 - val_loss: 0.0974


### Make Predictions

In [ ]:
input_text = vectorizer(['aku mau masak nasi goreng'])

In [60]:
res = model.predict(input_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step


In [61]:
(res > 0.5).astype(int)

array([[0]])

In [62]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [63]:
(model.predict(batch_X) > 0.5).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step


array([[0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [64]:
res.shape

(1, 1)

## Evaluasi

In [65]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [66]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [67]:
for batch in test.as_numpy_iterator():
    X_true, y_true = batch

    # Prediksi
    yhat = model.predict(X_true)

    # Convert & reshape
    y_true = y_true.flatten()
    yhat = yhat.flatten()

    # Threshold prediksi
    yhat = np.where(yhat > 0.5, 1, 0)

    # Pastikan tipe data cocok
    y_true = y_true.astype('int32')
    yhat = yhat.astype('int32')

    # Update metrik
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step

In [68]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.968692421913147, Recall:0.9511753916740417, Accuracy:0.9664633870124817


In [69]:
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [70]:
model.export("model_tf")

INFO:tensorflow:Assets written to: model_tf\assets


INFO:tensorflow:Assets written to: model_tf\assets


Saved artifact at 'model_tf'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1800), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2065978774480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065978775632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065978774672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065978773712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065978772944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065969487632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065969488208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065969487056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065969486096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065969487824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2065969489360: TensorSpec(shape=(), dtype=tf.reso